<a href="https://colab.research.google.com/github/divaldosilva/projetof1/blob/main/Portifolio_F1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação do Ambiente

## Bibliotecas Gerais

In [ ]:
import pandas as pd
import json

import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta

## Get API

In [ ]:
!pip install requests
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Google Translator

In [ ]:
# https://pypi.org/project/googletrans/
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 1.4 MB 32.3 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 65 kB 1.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=af2fd33348176bc5bce3334bb51e29aad6abea58e7cb4eaf7c9311af134a32e0
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


## Spark

In [ ]:
# Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Download Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz


In [ ]:
# Descompactar Spark
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
# Instalar
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
#from pyspark.sql.functions import col, expr, when, sum, concat_ws
from pyspark.sql.functions import *

from pyspark.sql.types import *

#from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [ ]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

## Configuração

In [ ]:
pd.set_option('display.max_columns', None)

# Funções Gerais

## Obter Idade

In [ ]:
def retorna_idade(data1: datetime ,data2: datetime) -> int :
  retorno = relativedelta(data1,data2)
  #print("Dayes:",retorno.days," Months:",retorno.months," Years:", retorno.years)
  #return pd.DataFrame([retorno.years,retorno.months,retorno.days],index=['anos','meses','dias']).transpose()
  return retorno.years


# Obter Dados

## [API] Temporadas


In [ ]:
# Obter da API
response = requests.get("http://ergast.com/api/f1/seasons.json?limit=1000")
get_temporadas = response.json()['MRData']['SeasonTable']['Seasons']
print(get_temporadas)

[{'season': '1950', 'url': 'http://en.wikipedia.org/wiki/1950_Formula_One_season'}, {'season': '1951', 'url': 'http://en.wikipedia.org/wiki/1951_Formula_One_season'}, {'season': '1952', 'url': 'http://en.wikipedia.org/wiki/1952_Formula_One_season'}, {'season': '1953', 'url': 'http://en.wikipedia.org/wiki/1953_Formula_One_season'}, {'season': '1954', 'url': 'http://en.wikipedia.org/wiki/1954_Formula_One_season'}, {'season': '1955', 'url': 'http://en.wikipedia.org/wiki/1955_Formula_One_season'}, {'season': '1956', 'url': 'http://en.wikipedia.org/wiki/1956_Formula_One_season'}, {'season': '1957', 'url': 'http://en.wikipedia.org/wiki/1957_Formula_One_season'}, {'season': '1958', 'url': 'http://en.wikipedia.org/wiki/1958_Formula_One_season'}, {'season': '1959', 'url': 'http://en.wikipedia.org/wiki/1959_Formula_One_season'}, {'season': '1960', 'url': 'http://en.wikipedia.org/wiki/1960_Formula_One_season'}, {'season': '1961', 'url': 'http://en.wikipedia.org/wiki/1961_Formula_One_season'}, {'s

In [ ]:
# Transformando em DataFrame
temporadas = pd.DataFrame.from_dict(get_temporadas)
temporadas.head()

,season,url
0,1950,http://en.wikipedia.org/wiki/1950_Formula_One_...
1,1951,http://en.wikipedia.org/wiki/1951_Formula_One_...
2,1952,http://en.wikipedia.org/wiki/1952_Formula_One_...
3,1953,http://en.wikipedia.org/wiki/1953_Formula_One_...
4,1954,http://en.wikipedia.org/wiki/1954_Formula_One_...


## [API] Resultados Corrida


In [ ]:
resultados = pd.DataFrame()

for temporada in temporadas.season:  
  response = requests.get("https://ergast.com/api/f1/" + temporada + "/results.json?limit=2000")
  resultados = resultados.append(pd.DataFrame.from_dict(response.json()['MRData']['RaceTable']['Races']))

### Paises

In [ ]:
paises = pd.read_csv('https://raw.githubusercontent.com/divaldosilva/projetof1/main/paises.csv')
paises

,pais_en,pais_pt,nacionalidade_en,nacionalidade_pt
0,Afghanistan,Afeganistão,Afghan,afegão
1,Andorra,Andorra,Andorran,andorrano
2,Angola,Angola,Angolan,angolano
3,Antigua e Barbuda,Antígua e Barbuda,Antiguan/Barbudan,antiguano
4,Algeria,Argélia,Algerian,argelino
...,...,...,...,...
187,Vietnam,Vietnã,Vietnamese,vietnamita
188,Wales,País de Gales,Welsh,galês
189,Yemen,Iêmen,Yemeni,iemenita
190,Zambia,Zâmbia,Zambian,zambiano


## Gravar Stages

In [ ]:
temporadas.to_parquet('/content/drive/MyDrive/Projetos/BI/F1/stage/temporadas.parquet')

In [ ]:
resultados.to_parquet('/content/drive/MyDrive/Projetos/BI/F1/stage/resultados.parquet')

In [ ]:
paises.to_parquet('/content/drive/MyDrive/Projetos/BI/F1/stage/paises.parquet')

# Transformação dos Dados

## Leitura Stages

### Temporadas

In [ ]:
df_temporadas = pd.read_parquet('/content/drive/MyDrive/Projetos/BI/F1/stage/temporadas.parquet')
df_temporadas

,season,url
0,1950,http://en.wikipedia.org/wiki/1950_Formula_One_...
1,1951,http://en.wikipedia.org/wiki/1951_Formula_One_...
2,1952,http://en.wikipedia.org/wiki/1952_Formula_One_...
3,1953,http://en.wikipedia.org/wiki/1953_Formula_One_...
4,1954,http://en.wikipedia.org/wiki/1954_Formula_One_...
...,...,...
68,2018,http://en.wikipedia.org/wiki/2018_Formula_One_...
69,2019,http://en.wikipedia.org/wiki/2019_Formula_One_...
70,2020,http://en.wikipedia.org/wiki/2020_Formula_One_...
71,2021,http://en.wikipedia.org/wiki/2021_Formula_One_...


### Resultados Corrida

In [ ]:
df_evento = pd.read_parquet('/content/drive/MyDrive/Projetos/BI/F1/stage/resultados.parquet')
df_evento

,season,round,url,raceName,Circuit,date,Results,time
0,1950,1,http://en.wikipedia.org/wiki/1950_British_Gran...,British Grand Prix,"{'Location': {'country': 'UK', 'lat': '52.0786...",1950-05-13,"[{'Constructor': {'constructorId': 'alfa', 'na...",None
1,1950,2,http://en.wikipedia.org/wiki/1950_Monaco_Grand...,Monaco Grand Prix,"{'Location': {'country': 'Monaco', 'lat': '43....",1950-05-21,"[{'Constructor': {'constructorId': 'alfa', 'na...",None
2,1950,3,http://en.wikipedia.org/wiki/1950_Indianapolis...,Indianapolis 500,"{'Location': {'country': 'USA', 'lat': '39.795...",1950-05-30,[{'Constructor': {'constructorId': 'kurtis_kra...,None
3,1950,4,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...,Swiss Grand Prix,"{'Location': {'country': 'Switzerland', 'lat':...",1950-06-04,"[{'Constructor': {'constructorId': 'alfa', 'na...",None
4,1950,5,http://en.wikipedia.org/wiki/1950_Belgian_Gran...,Belgian Grand Prix,"{'Location': {'country': 'Belgium', 'lat': '50...",1950-06-18,"[{'Constructor': {'constructorId': 'alfa', 'na...",None
...,...,...,...,...,...,...,...,...
15,2022,16,http://en.wikipedia.org/wiki/2022_Italian_Gran...,Italian Grand Prix,"{'Location': {'country': 'Italy', 'lat': '45.6...",2022-09-11,"[{'Constructor': {'constructorId': 'red_bull',...",13:00:00Z
16,2022,17,http://en.wikipedia.org/wiki/2022_Singapore_Gr...,Singapore Grand Prix,"{'Location': {'country': 'Singapore', 'lat': '...",2022-10-02,"[{'Constructor': {'constructorId': 'red_bull',...",12:00:00Z
17,2022,18,http://en.wikipedia.org/wiki/2022_Japanese_Gra...,Japanese Grand Prix,"{'Location': {'country': 'Japan', 'lat': '34.8...",2022-10-09,"[{'Constructor': {'constructorId': 'red_bull',...",05:00:00Z
18,2022,19,http://en.wikipedia.org/wiki/2022_United_State...,United States Grand Prix,"{'Location': {'country': 'USA', 'lat': '30.132...",2022-10-23,"[{'Constructor': {'constructorId': 'red_bull',...",19:00:00Z


### Paises

In [ ]:
df_paises = pd.read_parquet('/content/drive/MyDrive/Projetos/BI/F1/stage/paises.parquet')
df_paises

,pais_en,pais_pt,nacionalidade_en,nacionalidade_pt
0,Afghanistan,Afeganistão,Afghan,afegão
1,Andorra,Andorra,Andorran,andorrano
2,Angola,Angola,Angolan,angolano
3,Antigua e Barbuda,Antígua e Barbuda,Antiguan/Barbudan,antiguano
4,Algeria,Argélia,Algerian,argelino
...,...,...,...,...
187,Vietnam,Vietnã,Vietnamese,vietnamita
188,Wales,País de Gales,Welsh,galês
189,Yemen,Iêmen,Yemeni,iemenita
190,Zambia,Zâmbia,Zambian,zambiano


## Transformação - Organização dos Dados

### Resultados Corrida - Geral

In [ ]:
df_json = df_evento.to_json(orient='records')
df_resultados_gerais = pd.json_normalize(json.loads(df_json),'Results', meta=['season','round'])

In [ ]:
df_resultados_gerais

,FastestLap,grid,laps,number,points,position,positionText,status,Constructor.constructorId,Constructor.name,Constructor.nationality,Constructor.url,Driver.code,Driver.dateOfBirth,Driver.driverId,Driver.familyName,Driver.givenName,Driver.nationality,Driver.permanentNumber,Driver.url,Time.millis,Time.time,Time,FastestLap.AverageSpeed.speed,FastestLap.AverageSpeed.units,FastestLap.Time.time,FastestLap.lap,FastestLap.rank,season,round
0,NaN,1,70,2,9,1,1,Finished,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,None,1906-10-30,farina,Farina,Nino,Italian,None,http://en.wikipedia.org/wiki/Nino_Farina,8003600,2:13:23.6,NaN,NaN,NaN,NaN,NaN,NaN,1950,1
1,NaN,2,70,3,6,2,2,Finished,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,None,1898-06-09,fagioli,Fagioli,Luigi,Italian,None,http://en.wikipedia.org/wiki/Luigi_Fagioli,8006200,+2.6,NaN,NaN,NaN,NaN,NaN,NaN,1950,1
2,NaN,4,70,4,4,3,3,Finished,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,None,1911-07-02,reg_parnell,Parnell,Reg,British,None,http://en.wikipedia.org/wiki/Reg_Parnell,8055600,+52.0,NaN,NaN,NaN,NaN,NaN,NaN,1950,1
3,NaN,6,68,14,3,4,4,+2 Laps,lago,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago,None,1904-10-08,cabantous,Cabantous,Yves,French,None,http://en.wikipedia.org/wiki/Yves_Giraud_Caban...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950,1
4,NaN,9,68,15,2,5,5,+2 Laps,lago,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago,None,1905-11-05,rosier,Rosier,Louis,French,None,http://en.wikipedia.org/wiki/Louis_Rosier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,NaN,15,70,47,0,16,16,+1 Lap,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team,MSC,1999-03-22,mick_schumacher,Schumacher,Mick,German,47,http://en.wikipedia.org/wiki/Mick_Schumacher,NaN,NaN,NaN,185.248,kph,1:23.641,30,19,2022,20
25796,NaN,19,70,20,0,17,17,+1 Lap,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team,MAG,1992-10-05,kevin_magnussen,Magnussen,Kevin,Danish,20,http://en.wikipedia.org/wiki/Kevin_Magnussen,NaN,NaN,NaN,186.007,kph,1:23.300,49,15,2022,20
25797,NaN,18,69,6,0,18,18,+2 Laps,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,LAT,1995-06-29,latifi,Latifi,Nicholas,Canadian,6,http://en.wikipedia.org/wiki/Nicholas_Latifi,NaN,NaN,NaN,185.098,kph,1:23.709,56,20,2022,20
25798,NaN,9,63,14,0,19,19,Engine,alpine,Alpine F1 Team,French,http://en.wikipedia.org/wiki/Alpine_F1_Team,ALO,1981-07-29,alonso,Alonso,Fernando,Spanish,14,http://en.wikipedia.org/wiki/Fernando_Alonso,NaN,NaN,NaN,186.981,kph,1:22.866,48,11,2022,20


### Paises

In [ ]:
df_paises.info()
df_paises

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   pais_en           192 non-null    object
 1   pais_pt           192 non-null    object
 2   nacionalidade_en  192 non-null    object
 3   nacionalidade_pt  191 non-null    object
dtypes: object(4)
memory usage: 6.1+ KB


,pais_en,pais_pt,nacionalidade_en,nacionalidade_pt
0,Afghanistan,Afeganistão,Afghan,afegão
1,Andorra,Andorra,Andorran,andorrano
2,Angola,Angola,Angolan,angolano
3,Antigua e Barbuda,Antígua e Barbuda,Antiguan/Barbudan,antiguano
4,Algeria,Argélia,Algerian,argelino
...,...,...,...,...
187,Vietnam,Vietnã,Vietnamese,vietnamita
188,Wales,País de Gales,Welsh,galês
189,Yemen,Iêmen,Yemeni,iemenita
190,Zambia,Zâmbia,Zambian,zambiano


In [ ]:
# Preenchendo valores nulos 
df_paises = df_paises.fillna('')

In [ ]:
# Convertendo para String
#df_paises = df_paises.applymap(str)

In [ ]:
# Todas informações em maisculo
df_paises = df_paises.apply(lambda x: x.astype(str).str.upper())

In [ ]:
df_paises['pais_id'] = df_paises.index + 1

In [ ]:
df_paises = df_paises.set_index('pais_id')

In [ ]:
# Verificando resultado
df_paises.info()
df_paises

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 1 to 192
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   pais_en           192 non-null    object
 1   pais_pt           192 non-null    object
 2   nacionalidade_en  192 non-null    object
 3   nacionalidade_pt  192 non-null    object
dtypes: object(4)
memory usage: 7.5+ KB


,pais_en,pais_pt,nacionalidade_en,nacionalidade_pt
pais_id,,,,
1,AFGHANISTAN,AFEGANISTÃO,AFGHAN,AFEGÃO
2,ANDORRA,ANDORRA,ANDORRAN,ANDORRANO
3,ANGOLA,ANGOLA,ANGOLAN,ANGOLANO
4,ANTIGUA E BARBUDA,ANTÍGUA E BARBUDA,ANTIGUAN/BARBUDAN,ANTIGUANO
5,ALGERIA,ARGÉLIA,ALGERIAN,ARGELINO
...,...,...,...,...
188,VIETNAM,VIETNÃ,VIETNAMESE,VIETNAMITA
189,WALES,PAÍS DE GALES,WELSH,GALÊS
190,YEMEN,IÊMEN,YEMENI,IEMENITA


### Situação Corrida

In [ ]:
df_situacao = pd.DataFrame()
# Obtendo os dados
df_situacao['situacao'] = df_resultados_gerais['status']

In [ ]:
# Convertendo para Data Frame
df_situacao = pd.DataFrame(df_situacao)

In [ ]:
# Eliminando os duplicados
df_situacao = df_situacao.drop_duplicates()

In [ ]:
# Reindexando
df_situacao.reset_index(inplace=True, drop=True)

In [ ]:
# Tradução
df_situacao['situacao_pt'] = df_situacao.apply(lambda row: translator.translate(row['situacao'], src='en', dest='pt').text,axis = 1)

In [ ]:
# Todas informações em maisculo
df_situacao = df_situacao.apply(lambda x: x.astype(str).str.upper())

In [ ]:
df_situacao['situacao_id'] = df_situacao.index + 1

In [ ]:
# Verificando resultado
df_situacao.info()
df_situacao

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   situacao     137 non-null    object
 1   situacao_pt  137 non-null    object
 2   situacao_id  137 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.3+ KB


,situacao,situacao_pt,situacao_id
0,FINISHED,FINALIZADO,1
1,+2 LAPS,+2 VOLTAS,2
2,+3 LAPS,+3 VOLTAS,3
3,+5 LAPS,+5 VOLTAS,4
4,+6 LAPS,+6 VOLTAS,5
...,...,...,...
132,DAMAGE,DANO,133
133,DEBRIS,DESTROÇOS,134
134,ILLNESS,DOENÇA,135
135,COOLING SYSTEM,SISTEMA DE REFRIGERAÇÃO,136


### Temporadas

In [ ]:
# Renomear colunas
df_temporadas = df_temporadas.rename(columns={'season' :'temporada',
                                                'url' : 'url_wiki'}
                                     )
df_temporadas.head(5)

,temporada,url_wiki
0,1950,http://en.wikipedia.org/wiki/1950_Formula_One_...
1,1951,http://en.wikipedia.org/wiki/1951_Formula_One_...
2,1952,http://en.wikipedia.org/wiki/1952_Formula_One_...
3,1953,http://en.wikipedia.org/wiki/1953_Formula_One_...
4,1954,http://en.wikipedia.org/wiki/1954_Formula_One_...


In [ ]:
df_temporadas['temporada'] = df_temporadas['temporada'].astype(int)

In [ ]:
df_temporadas['temporada_id'] = df_temporadas.index + 1

In [ ]:
# Verificando resultado
df_temporadas.info()
df_temporadas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   temporada     73 non-null     int64 
 1   url_wiki      73 non-null     object
 2   temporada_id  73 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ KB


,temporada,url_wiki,temporada_id
0,1950,http://en.wikipedia.org/wiki/1950_Formula_One_...,1
1,1951,http://en.wikipedia.org/wiki/1951_Formula_One_...,2
2,1952,http://en.wikipedia.org/wiki/1952_Formula_One_...,3
3,1953,http://en.wikipedia.org/wiki/1953_Formula_One_...,4
4,1954,http://en.wikipedia.org/wiki/1954_Formula_One_...,5
...,...,...,...
68,2018,http://en.wikipedia.org/wiki/2018_Formula_One_...,69
69,2019,http://en.wikipedia.org/wiki/2019_Formula_One_...,70
70,2020,http://en.wikipedia.org/wiki/2020_Formula_One_...,71
71,2021,http://en.wikipedia.org/wiki/2021_Formula_One_...,72


### Circuitos

In [ ]:
# Obtendo circuitos
df_circuito = pd.json_normalize(df_evento['Circuit'])

df_circuito.head(5)

,circuitId,circuitName,url,Location.country,Location.lat,Location.locality,Location.long
0,silverstone,Silverstone Circuit,http://en.wikipedia.org/wiki/Silverstone_Circuit,UK,52.0786,Silverstone,-1.01694
1,monaco,Circuit de Monaco,http://en.wikipedia.org/wiki/Circuit_de_Monaco,Monaco,43.7347,Monte-Carlo,7.42056
2,indianapolis,Indianapolis Motor Speedway,http://en.wikipedia.org/wiki/Indianapolis_Moto...,USA,39.795,Indianapolis,-86.2347
3,bremgarten,Circuit Bremgarten,http://en.wikipedia.org/wiki/Circuit_Bremgarten,Switzerland,46.9589,Bern,7.40194
4,spa,Circuit de Spa-Francorchamps,http://en.wikipedia.org/wiki/Circuit_de_Spa-Fr...,Belgium,50.4372,Spa,5.97139


In [ ]:
# Renomear colunas
df_circuito = df_circuito.rename(columns={'circuitId' :'circuito',
                                          'circuitName' : 'circuito_nome',
                                          'url':'url_wiki',
                                          'Location.country' : 'pais_en',
                                          'Location.lat' : 'latitute',
                                          'Location.long' : 'longitude',
                                          'Location.locality' : 'localizacao'
                                          }
                                  )
df_circuito.head(5)

,circuito,circuito_nome,url_wiki,pais_en,latitute,localizacao,longitude
0,silverstone,Silverstone Circuit,http://en.wikipedia.org/wiki/Silverstone_Circuit,UK,52.0786,Silverstone,-1.01694
1,monaco,Circuit de Monaco,http://en.wikipedia.org/wiki/Circuit_de_Monaco,Monaco,43.7347,Monte-Carlo,7.42056
2,indianapolis,Indianapolis Motor Speedway,http://en.wikipedia.org/wiki/Indianapolis_Moto...,USA,39.795,Indianapolis,-86.2347
3,bremgarten,Circuit Bremgarten,http://en.wikipedia.org/wiki/Circuit_Bremgarten,Switzerland,46.9589,Bern,7.40194
4,spa,Circuit de Spa-Francorchamps,http://en.wikipedia.org/wiki/Circuit_de_Spa-Fr...,Belgium,50.4372,Spa,5.97139


In [ ]:
# Eliminar duplicadas
t_ant = len(df_circuito)
df_circuito = df_circuito.drop_duplicates()

print('Antes: ', t_ant ,' / Depois:' , len(df_circuito))
df_circuito

Antes:  1077  / Depois: 76


,circuito,circuito_nome,url_wiki,pais_en,latitute,localizacao,longitude
0,silverstone,Silverstone Circuit,http://en.wikipedia.org/wiki/Silverstone_Circuit,UK,52.0786,Silverstone,-1.01694
1,monaco,Circuit de Monaco,http://en.wikipedia.org/wiki/Circuit_de_Monaco,Monaco,43.7347,Monte-Carlo,7.42056
2,indianapolis,Indianapolis Motor Speedway,http://en.wikipedia.org/wiki/Indianapolis_Moto...,USA,39.795,Indianapolis,-86.2347
3,bremgarten,Circuit Bremgarten,http://en.wikipedia.org/wiki/Circuit_Bremgarten,Switzerland,46.9589,Bern,7.40194
4,spa,Circuit de Spa-Francorchamps,http://en.wikipedia.org/wiki/Circuit_de_Spa-Fr...,Belgium,50.4372,Spa,5.97139
...,...,...,...,...,...,...,...
1026,mugello,Autodromo Internazionale del Mugello,http://en.wikipedia.org/wiki/Mugello_Circuit,Italy,43.9975,Mugello,11.3719
1029,portimao,Autódromo Internacional do Algarve,http://en.wikipedia.org/wiki/Algarve_Internati...,Portugal,37.227,Portimão,-8.6267
1054,losail,Losail International Circuit,http://en.wikipedia.org/wiki/Losail_Internatio...,Qatar,25.49,Al Daayen,51.4542
1055,jeddah,Jeddah Corniche Circuit,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...,Saudi Arabia,21.6319,Jeddah,39.1044


In [ ]:
# Todas informações em maisculo
df_circuito = df_circuito.apply(lambda x: x.astype(str).str.upper())
df_circuito['circuito'] = df_circuito['circuito'].astype(str).str.lower()
df_circuito['url_wiki'] = df_circuito['url_wiki'].astype(str).str.lower()



In [ ]:
df_circuito.reset_index(drop=True, inplace=True)
df_circuito['circuito_id'] = df_circuito.index + 1
#df_circuito = df_circuito.set_index('circuito_id')

In [ ]:
# Verificando resultado
df_circuito.info()
df_circuito

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   circuito       76 non-null     object
 1   circuito_nome  76 non-null     object
 2   url_wiki       76 non-null     object
 3   pais_en        76 non-null     object
 4   latitute       76 non-null     object
 5   localizacao    76 non-null     object
 6   longitude      76 non-null     object
 7   circuito_id    76 non-null     int64 
dtypes: int64(1), object(7)
memory usage: 4.9+ KB


,circuito,circuito_nome,url_wiki,pais_en,latitute,localizacao,longitude,circuito_id
0,silverstone,SILVERSTONE CIRCUIT,http://en.wikipedia.org/wiki/silverstone_circuit,UK,52.0786,SILVERSTONE,-1.01694,1
1,monaco,CIRCUIT DE MONACO,http://en.wikipedia.org/wiki/circuit_de_monaco,MONACO,43.7347,MONTE-CARLO,7.42056,2
2,indianapolis,INDIANAPOLIS MOTOR SPEEDWAY,http://en.wikipedia.org/wiki/indianapolis_moto...,USA,39.795,INDIANAPOLIS,-86.2347,3
3,bremgarten,CIRCUIT BREMGARTEN,http://en.wikipedia.org/wiki/circuit_bremgarten,SWITZERLAND,46.9589,BERN,7.40194,4
4,spa,CIRCUIT DE SPA-FRANCORCHAMPS,http://en.wikipedia.org/wiki/circuit_de_spa-fr...,BELGIUM,50.4372,SPA,5.97139,5
...,...,...,...,...,...,...,...,...
71,mugello,AUTODROMO INTERNAZIONALE DEL MUGELLO,http://en.wikipedia.org/wiki/mugello_circuit,ITALY,43.9975,MUGELLO,11.3719,72
72,portimao,AUTÓDROMO INTERNACIONAL DO ALGARVE,http://en.wikipedia.org/wiki/algarve_internati...,PORTUGAL,37.227,PORTIMÃO,-8.6267,73
73,losail,LOSAIL INTERNATIONAL CIRCUIT,http://en.wikipedia.org/wiki/losail_internatio...,QATAR,25.49,AL DAAYEN,51.4542,74
74,jeddah,JEDDAH CORNICHE CIRCUIT,http://en.wikipedia.org/wiki/jeddah_street_cir...,SAUDI ARABIA,21.6319,JEDDAH,39.1044,75


### Construtores

In [ ]:
# Obtendo dados de Construtores
df_construtores = pd.DataFrame()

df_construtores['construtor'] = df_resultados_gerais['Constructor.constructorId']
df_construtores['nome'] = df_resultados_gerais['Constructor.name']
df_construtores['nacionalidade_en'] = df_resultados_gerais['Constructor.nationality']
df_construtores['url_wiki'] = df_resultados_gerais['Constructor.url']

df_construtores


,construtor,nome,nacionalidade_en,url_wiki
0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
1,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
2,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
3,lago,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago
4,lago,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago
...,...,...,...,...
25795,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
25796,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
25797,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
25798,alpine,Alpine F1 Team,French,http://en.wikipedia.org/wiki/Alpine_F1_Team


In [ ]:
# Eliminar duplicadas
t_ant = len(df_construtores)
df_construtores = df_construtores.drop_duplicates()

print('Antes: ', t_ant ,' / Depois:' , len(df_construtores))

Antes:  25800  / Depois: 210


In [ ]:
# Todas informações em maisculo
df_construtores = df_construtores.apply(lambda x: x.astype(str).str.upper())
df_construtores['construtor'] = df_construtores['construtor'].astype(str).str.lower()
df_construtores['url_wiki'] = df_construtores['url_wiki'].astype(str).str.lower()

In [ ]:
# Criando Indices
df_construtores.reset_index(drop=True, inplace=True)
df_construtores['construtor_id'] = df_construtores.index + 1
#df_construtores = df_construtores.set_index('construtor_id')

In [ ]:
# Verificando resultado
df_construtores.info()
df_construtores

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   construtor        210 non-null    object
 1   nome              210 non-null    object
 2   nacionalidade_en  210 non-null    object
 3   url_wiki          210 non-null    object
 4   construtor_id     210 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 8.3+ KB


,construtor,nome,nacionalidade_en,url_wiki,construtor_id
0,alfa,ALFA ROMEO,SWISS,http://en.wikipedia.org/wiki/alfa_romeo_in_for...,1
1,lago,TALBOT-LAGO,FRENCH,http://en.wikipedia.org/wiki/talbot-lago,2
2,era,ERA,BRITISH,http://en.wikipedia.org/wiki/english_racing_au...,3
3,maserati,MASERATI,ITALIAN,http://en.wikipedia.org/wiki/maserati,4
4,alta,ALTA,BRITISH,http://en.wikipedia.org/wiki/alta_auto_racing_...,5
...,...,...,...,...,...
205,manor,MANOR MARUSSIA,BRITISH,http://en.wikipedia.org/wiki/manor_motorsport,206
206,haas,HAAS F1 TEAM,AMERICAN,http://en.wikipedia.org/wiki/haas_f1_team,207
207,racing_point,RACING POINT,BRITISH,http://en.wikipedia.org/wiki/racing_point_f1_team,208
208,alphatauri,ALPHATAURI,ITALIAN,http://en.wikipedia.org/wiki/scuderia_alphatauri,209


### Pilotos

In [ ]:
# Obtendo dados de Pilotos
df_pilotos = pd.DataFrame()

df_pilotos['codigo'] = df_resultados_gerais['Driver.code']
df_pilotos['nascimento'] = df_resultados_gerais['Driver.dateOfBirth']
df_pilotos['piloto'] = df_resultados_gerais['Driver.driverId']
df_pilotos['sobrenome'] = df_resultados_gerais['Driver.familyName']
df_pilotos['nome'] = df_resultados_gerais['Driver.givenName']
df_pilotos['nacionalidade_en'] = df_resultados_gerais['Driver.nationality']
df_pilotos['numero_carro'] = df_resultados_gerais['Driver.permanentNumber']
df_pilotos['url_wiki'] = df_resultados_gerais['Driver.url']

df_pilotos

,codigo,nascimento,piloto,sobrenome,nome,nacionalidade_en,numero_carro,url_wiki
0,None,1906-10-30,farina,Farina,Nino,Italian,None,http://en.wikipedia.org/wiki/Nino_Farina
1,None,1898-06-09,fagioli,Fagioli,Luigi,Italian,None,http://en.wikipedia.org/wiki/Luigi_Fagioli
2,None,1911-07-02,reg_parnell,Parnell,Reg,British,None,http://en.wikipedia.org/wiki/Reg_Parnell
3,None,1904-10-08,cabantous,Cabantous,Yves,French,None,http://en.wikipedia.org/wiki/Yves_Giraud_Caban...
4,None,1905-11-05,rosier,Rosier,Louis,French,None,http://en.wikipedia.org/wiki/Louis_Rosier
...,...,...,...,...,...,...,...,...
25795,MSC,1999-03-22,mick_schumacher,Schumacher,Mick,German,47,http://en.wikipedia.org/wiki/Mick_Schumacher
25796,MAG,1992-10-05,kevin_magnussen,Magnussen,Kevin,Danish,20,http://en.wikipedia.org/wiki/Kevin_Magnussen
25797,LAT,1995-06-29,latifi,Latifi,Nicholas,Canadian,6,http://en.wikipedia.org/wiki/Nicholas_Latifi
25798,ALO,1981-07-29,alonso,Alonso,Fernando,Spanish,14,http://en.wikipedia.org/wiki/Fernando_Alonso


In [ ]:
# Eliminar duplicadas
t_ant = len(df_pilotos)
df_pilotos = df_pilotos.drop_duplicates()

print('Antes: ', t_ant ,' / Depois:' , len(df_pilotos))

Antes:  25800  / Depois: 855


In [ ]:
# Todas informações em maisculo
df_pilotos = df_pilotos.apply(lambda x: x.astype(str).str.upper())
df_pilotos['piloto'] = df_pilotos['piloto'].astype(str).str.lower()
df_pilotos['url_wiki'] = df_pilotos['url_wiki'].astype(str).str.lower()

df_pilotos

,codigo,nascimento,piloto,sobrenome,nome,nacionalidade_en,numero_carro,url_wiki
0,NONE,1906-10-30,farina,FARINA,NINO,ITALIAN,NONE,http://en.wikipedia.org/wiki/nino_farina
1,NONE,1898-06-09,fagioli,FAGIOLI,LUIGI,ITALIAN,NONE,http://en.wikipedia.org/wiki/luigi_fagioli
2,NONE,1911-07-02,reg_parnell,PARNELL,REG,BRITISH,NONE,http://en.wikipedia.org/wiki/reg_parnell
3,NONE,1904-10-08,cabantous,CABANTOUS,YVES,FRENCH,NONE,http://en.wikipedia.org/wiki/yves_giraud_caban...
4,NONE,1905-11-05,rosier,ROSIER,LOUIS,FRENCH,NONE,http://en.wikipedia.org/wiki/louis_rosier
...,...,...,...,...,...,...,...,...
24968,TSU,2000-05-11,tsunoda,TSUNODA,YUKI,JAPANESE,22,http://en.wikipedia.org/wiki/yuki_tsunoda
24975,MSC,1999-03-22,mick_schumacher,SCHUMACHER,MICK,GERMAN,47,http://en.wikipedia.org/wiki/mick_schumacher
24979,MAZ,1999-03-02,mazepin,MAZEPIN,NIKITA,RUSSIAN,9,http://en.wikipedia.org/wiki/nikita_mazepin
25409,ZHO,1999-05-30,zhou,ZHOU,GUANYU,CHINESE,24,http://en.wikipedia.org/wiki/guanyu_zhou


In [ ]:
# Ajustando valores
df_pilotos['codigo'].replace('NONE','',inplace=True)
df_pilotos['numero_carro'].replace('NONE','',inplace=True)

In [ ]:
# Criando Indices
df_pilotos.reset_index(drop=True, inplace=True)
df_pilotos['piloto_id'] = df_pilotos.index + 1
#df_pilotos = df_pilotos.set_index('piloto_id')

In [ ]:
# Verificando resultado
df_pilotos.info()
df_pilotos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855 entries, 0 to 854
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   codigo            855 non-null    object
 1   nascimento        855 non-null    object
 2   piloto            855 non-null    object
 3   sobrenome         855 non-null    object
 4   nome              855 non-null    object
 5   nacionalidade_en  855 non-null    object
 6   numero_carro      855 non-null    object
 7   url_wiki          855 non-null    object
 8   piloto_id         855 non-null    int64 
dtypes: int64(1), object(8)
memory usage: 60.2+ KB


,codigo,nascimento,piloto,sobrenome,nome,nacionalidade_en,numero_carro,url_wiki,piloto_id
0,,1906-10-30,farina,FARINA,NINO,ITALIAN,,http://en.wikipedia.org/wiki/nino_farina,1
1,,1898-06-09,fagioli,FAGIOLI,LUIGI,ITALIAN,,http://en.wikipedia.org/wiki/luigi_fagioli,2
2,,1911-07-02,reg_parnell,PARNELL,REG,BRITISH,,http://en.wikipedia.org/wiki/reg_parnell,3
3,,1904-10-08,cabantous,CABANTOUS,YVES,FRENCH,,http://en.wikipedia.org/wiki/yves_giraud_caban...,4
4,,1905-11-05,rosier,ROSIER,LOUIS,FRENCH,,http://en.wikipedia.org/wiki/louis_rosier,5
...,...,...,...,...,...,...,...,...,...
850,TSU,2000-05-11,tsunoda,TSUNODA,YUKI,JAPANESE,22,http://en.wikipedia.org/wiki/yuki_tsunoda,851
851,MSC,1999-03-22,mick_schumacher,SCHUMACHER,MICK,GERMAN,47,http://en.wikipedia.org/wiki/mick_schumacher,852
852,MAZ,1999-03-02,mazepin,MAZEPIN,NIKITA,RUSSIAN,9,http://en.wikipedia.org/wiki/nikita_mazepin,853
853,ZHO,1999-05-30,zhou,ZHOU,GUANYU,CHINESE,24,http://en.wikipedia.org/wiki/guanyu_zhou,854


### Etapas

In [ ]:
# Obtendo dados de Corridas
df_etapas = pd.DataFrame()

df_etapas['temporada'] = df_evento['season']
df_etapas['etapa'] = df_evento['round']
df_etapas['url_wiki'] = df_evento['url']
df_etapas['nome'] = df_evento['raceName']
df_etapas['data'] = df_evento['date']
df_etapas['hora'] = df_evento['time']
df_etapas['circuito'] = df_evento['Circuit'].apply(pd.Series).circuitId

df_etapas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1077 entries, 0 to 19
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   temporada  1077 non-null   object
 1   etapa      1077 non-null   object
 2   url_wiki   1077 non-null   object
 3   nome       1077 non-null   object
 4   data       1077 non-null   object
 5   hora       346 non-null    object
 6   circuito   1077 non-null   object
dtypes: object(7)
memory usage: 99.6+ KB


##### Tratamento dos Campos

In [ ]:
# Tratando informações NULL
df_etapas = df_etapas.fillna('')

In [ ]:
# Todas informações em maisculo
df_etapas = df_etapas.apply(lambda x: x.astype(str).str.upper())
df_etapas['circuito'] = df_etapas['circuito'].astype(str).str.lower()
df_etapas['url_wiki'] = df_etapas['url_wiki'].astype(str).str.lower()

In [ ]:
# Tipando os campos
df_etapas['etapa'] = df_etapas['etapa'].astype(int)
df_etapas['temporada'] = df_etapas['temporada'].astype(int)

df_etapas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1077 entries, 0 to 19
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   temporada  1077 non-null   int64 
 1   etapa      1077 non-null   int64 
 2   url_wiki   1077 non-null   object
 3   nome       1077 non-null   object
 4   data       1077 non-null   object
 5   hora       1077 non-null   object
 6   circuito   1077 non-null   object
dtypes: int64(2), object(5)
memory usage: 99.6+ KB


##### Obtendo chaves estrangeiras

In [ ]:
# Obtendo ID da Temporada
df_etapas = df_etapas.merge(df_temporadas[['temporada','temporada_id']],how='inner', on='temporada')
df_etapas
df_etapas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1077 entries, 0 to 1076
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   temporada     1077 non-null   int64 
 1   etapa         1077 non-null   int64 
 2   url_wiki      1077 non-null   object
 3   nome          1077 non-null   object
 4   data          1077 non-null   object
 5   hora          1077 non-null   object
 6   circuito      1077 non-null   object
 7   temporada_id  1077 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 75.7+ KB


In [ ]:
# Obtendo ID do circuito
df_etapas = df_etapas.merge(df_circuito[['circuito','circuito_id']],how='inner', on='circuito')
df_etapas

,temporada,etapa,url_wiki,nome,data,hora,circuito,temporada_id,circuito_id
0,1950,1,http://en.wikipedia.org/wiki/1950_british_gran...,BRITISH GRAND PRIX,1950-05-13,,silverstone,1,1
1,1951,5,http://en.wikipedia.org/wiki/1951_british_gran...,BRITISH GRAND PRIX,1951-07-14,,silverstone,2,1
2,1952,5,http://en.wikipedia.org/wiki/1952_british_gran...,BRITISH GRAND PRIX,1952-07-19,,silverstone,3,1
3,1953,6,http://en.wikipedia.org/wiki/1953_british_gran...,BRITISH GRAND PRIX,1953-07-18,,silverstone,4,1
4,1954,5,http://en.wikipedia.org/wiki/1954_british_gran...,BRITISH GRAND PRIX,1954-07-17,,silverstone,5,1
...,...,...,...,...,...,...,...,...,...
1072,2021,3,http://en.wikipedia.org/wiki/2021_portuguese_g...,PORTUGUESE GRAND PRIX,2021-05-02,14:00:00Z,portimao,72,73
1073,2021,20,http://en.wikipedia.org/wiki/2021_qatar_grand_...,QATAR GRAND PRIX,2021-11-21,14:00:00Z,losail,72,74
1074,2021,21,http://en.wikipedia.org/wiki/2021_saudi_arabia...,SAUDI ARABIAN GRAND PRIX,2021-12-05,17:30:00Z,jeddah,72,75
1075,2022,2,http://en.wikipedia.org/wiki/2022_saudi_arabia...,SAUDI ARABIAN GRAND PRIX,2022-03-27,17:00:00Z,jeddah,73,75


##### Obtendo nome do GP

In [ ]:
# Tradução do nome do GP
df_gp = pd.DataFrame()
df_gp['nome'] = df_etapas['nome']
df_gp = df_gp.drop_duplicates()

df_gp['nome_pt'] = df_gp.apply(lambda row: translator.translate(row['nome'], src='en', dest='pt').text,axis = 1)
df_gp.count()

nome       53
nome_pt    53
dtype: int64

In [ ]:
# Join com o nome de GP traduzido
df_etapas = df_etapas.merge(df_gp[['nome','nome_pt']],how='left', on='nome')
df_etapas

,temporada,etapa,url_wiki,nome,data,hora,circuito,temporada_id,circuito_id,nome_pt
0,1950,1,http://en.wikipedia.org/wiki/1950_british_gran...,BRITISH GRAND PRIX,1950-05-13,,silverstone,1,1,GRANDE PRÊMIO BRITÂNICO
1,1951,5,http://en.wikipedia.org/wiki/1951_british_gran...,BRITISH GRAND PRIX,1951-07-14,,silverstone,2,1,GRANDE PRÊMIO BRITÂNICO
2,1952,5,http://en.wikipedia.org/wiki/1952_british_gran...,BRITISH GRAND PRIX,1952-07-19,,silverstone,3,1,GRANDE PRÊMIO BRITÂNICO
3,1953,6,http://en.wikipedia.org/wiki/1953_british_gran...,BRITISH GRAND PRIX,1953-07-18,,silverstone,4,1,GRANDE PRÊMIO BRITÂNICO
4,1954,5,http://en.wikipedia.org/wiki/1954_british_gran...,BRITISH GRAND PRIX,1954-07-17,,silverstone,5,1,GRANDE PRÊMIO BRITÂNICO
...,...,...,...,...,...,...,...,...,...,...
1072,2021,3,http://en.wikipedia.org/wiki/2021_portuguese_g...,PORTUGUESE GRAND PRIX,2021-05-02,14:00:00Z,portimao,72,73,GRANDE PRÉMIO PORTUGUÊS
1073,2021,20,http://en.wikipedia.org/wiki/2021_qatar_grand_...,QATAR GRAND PRIX,2021-11-21,14:00:00Z,losail,72,74,GRANDE PRÊMIO DO CATAR
1074,2021,21,http://en.wikipedia.org/wiki/2021_saudi_arabia...,SAUDI ARABIAN GRAND PRIX,2021-12-05,17:30:00Z,jeddah,72,75,GRANDE PRÊMIO DA ÁRABE SAUDITA
1075,2022,2,http://en.wikipedia.org/wiki/2022_saudi_arabia...,SAUDI ARABIAN GRAND PRIX,2022-03-27,17:00:00Z,jeddah,73,75,GRANDE PRÊMIO DA ÁRABE SAUDITA


##### Criando campo chave

In [ ]:
# Criando campo chave
df_etapas = df_etapas.sort_values(by=['temporada','etapa'], ascending=True)
df_etapas.reset_index(drop=True, inplace=True)
df_etapas['etapa_id'] = df_etapas.index + 1
df_etapas


,temporada,etapa,url_wiki,nome,data,hora,circuito,temporada_id,circuito_id,nome_pt,etapa_id
0,1950,1,http://en.wikipedia.org/wiki/1950_british_gran...,BRITISH GRAND PRIX,1950-05-13,,silverstone,1,1,GRANDE PRÊMIO BRITÂNICO,1
1,1950,2,http://en.wikipedia.org/wiki/1950_monaco_grand...,MONACO GRAND PRIX,1950-05-21,,monaco,1,2,GRANDE PRÊMIO DE MÔNACO,2
2,1950,3,http://en.wikipedia.org/wiki/1950_indianapolis...,INDIANAPOLIS 500,1950-05-30,,indianapolis,1,3,INDIANÁPOLIS 500,3
3,1950,4,http://en.wikipedia.org/wiki/1950_swiss_grand_...,SWISS GRAND PRIX,1950-06-04,,bremgarten,1,4,GRANDE PRÊMIO DA SUÍÇA,4
4,1950,5,http://en.wikipedia.org/wiki/1950_belgian_gran...,BELGIAN GRAND PRIX,1950-06-18,,spa,1,5,GRANDE PRÊMIO DA BELGA,5
...,...,...,...,...,...,...,...,...,...,...,...
1072,2022,16,http://en.wikipedia.org/wiki/2022_italian_gran...,ITALIAN GRAND PRIX,2022-09-11,13:00:00Z,monza,73,7,GRANDE PRÊMIO ITALIANO,1073
1073,2022,17,http://en.wikipedia.org/wiki/2022_singapore_gr...,SINGAPORE GRAND PRIX,2022-10-02,12:00:00Z,marina_bay,73,65,GRANDE PRÊMIO DE CINGAPURA,1074
1074,2022,18,http://en.wikipedia.org/wiki/2022_japanese_gra...,JAPANESE GRAND PRIX,2022-10-09,05:00:00Z,suzuka,73,53,GRANDE PRÊMIO DO JAPONÊS,1075
1075,2022,19,http://en.wikipedia.org/wiki/2022_united_state...,UNITED STATES GRAND PRIX,2022-10-23,19:00:00Z,americas,73,69,GRANDE PRÊMIO DOS ESTADOS UNIDOS,1076


##### Organização das colunas

In [ ]:
# Organizando Colunas
df_etapas = df_etapas[['etapa_id','temporada_id','etapa','circuito_id','nome','nome_pt','data','hora']]
df_etapas

,etapa_id,temporada_id,etapa,circuito_id,nome,nome_pt,data,hora
0,1,1,1,1,BRITISH GRAND PRIX,GRANDE PRÊMIO BRITÂNICO,1950-05-13,
1,2,1,2,2,MONACO GRAND PRIX,GRANDE PRÊMIO DE MÔNACO,1950-05-21,
2,3,1,3,3,INDIANAPOLIS 500,INDIANÁPOLIS 500,1950-05-30,
3,4,1,4,4,SWISS GRAND PRIX,GRANDE PRÊMIO DA SUÍÇA,1950-06-04,
4,5,1,5,5,BELGIAN GRAND PRIX,GRANDE PRÊMIO DA BELGA,1950-06-18,
...,...,...,...,...,...,...,...,...
1072,1073,73,16,7,ITALIAN GRAND PRIX,GRANDE PRÊMIO ITALIANO,2022-09-11,13:00:00Z
1073,1074,73,17,65,SINGAPORE GRAND PRIX,GRANDE PRÊMIO DE CINGAPURA,2022-10-02,12:00:00Z
1074,1075,73,18,53,JAPANESE GRAND PRIX,GRANDE PRÊMIO DO JAPONÊS,2022-10-09,05:00:00Z
1075,1076,73,19,69,UNITED STATES GRAND PRIX,GRANDE PRÊMIO DOS ESTADOS UNIDOS,2022-10-23,19:00:00Z


In [ ]:
# Verificando resultado
df_etapas.info()
df_etapas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077 entries, 0 to 1076
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   etapa_id      1077 non-null   int64 
 1   temporada_id  1077 non-null   int64 
 2   etapa         1077 non-null   int64 
 3   circuito_id   1077 non-null   int64 
 4   nome          1077 non-null   object
 5   nome_pt       1077 non-null   object
 6   data          1077 non-null   object
 7   hora          1077 non-null   object
dtypes: int64(4), object(4)
memory usage: 67.4+ KB


,etapa_id,temporada_id,etapa,circuito_id,nome,nome_pt,data,hora
0,1,1,1,1,BRITISH GRAND PRIX,GRANDE PRÊMIO BRITÂNICO,1950-05-13,
1,2,1,2,2,MONACO GRAND PRIX,GRANDE PRÊMIO DE MÔNACO,1950-05-21,
2,3,1,3,3,INDIANAPOLIS 500,INDIANÁPOLIS 500,1950-05-30,
3,4,1,4,4,SWISS GRAND PRIX,GRANDE PRÊMIO DA SUÍÇA,1950-06-04,
4,5,1,5,5,BELGIAN GRAND PRIX,GRANDE PRÊMIO DA BELGA,1950-06-18,
...,...,...,...,...,...,...,...,...
1072,1073,73,16,7,ITALIAN GRAND PRIX,GRANDE PRÊMIO ITALIANO,2022-09-11,13:00:00Z
1073,1074,73,17,65,SINGAPORE GRAND PRIX,GRANDE PRÊMIO DE CINGAPURA,2022-10-02,12:00:00Z
1074,1075,73,18,53,JAPANESE GRAND PRIX,GRANDE PRÊMIO DO JAPONÊS,2022-10-09,05:00:00Z
1075,1076,73,19,69,UNITED STATES GRAND PRIX,GRANDE PRÊMIO DOS ESTADOS UNIDOS,2022-10-23,19:00:00Z


### Resultados

In [ ]:
# Obtendo dados dos Resultados da Corrida
df_resultados_corrida = pd.DataFrame()

# Chaves
df_resultados_corrida['temporada'] = df_resultados_gerais['season'].astype(int)
df_resultados_corrida['etapa'] = df_resultados_gerais['round'].astype(int)
df_resultados_corrida['piloto'] = df_resultados_gerais['Driver.driverId'].astype('string')
df_resultados_corrida['construtor'] = df_resultados_gerais['Constructor.constructorId'].astype('string')

# Resultado
df_resultados_corrida['numero'] = df_resultados_gerais['number'].astype('string')
df_resultados_corrida['largada'] = df_resultados_gerais['grid'].astype(int)
df_resultados_corrida['voltas'] = df_resultados_gerais['laps'].astype(int)
df_resultados_corrida['posicao'] = df_resultados_gerais['position'].astype(int)
df_resultados_corrida['posicao_t'] = df_resultados_gerais['positionText'].astype('string')
df_resultados_corrida['pontos'] = df_resultados_gerais['points'].astype(float)
df_resultados_corrida['situacao'] = df_resultados_gerais['status'].astype('string')

# Tempo Corrida
df_resultados_corrida['tempo_mili'] = df_resultados_gerais['Time.millis'].astype('string').fillna('', inplace=False)
df_resultados_corrida['tempo'] = df_resultados_gerais['Time.time'].astype('string').fillna('', inplace=False)

# Volta rapida
df_resultados_corrida['vr_media_velocidade'] = df_resultados_gerais['FastestLap.AverageSpeed.speed'].astype(float).fillna(0, inplace=False)
df_resultados_corrida['vr_media_unidade'] = df_resultados_gerais['FastestLap.AverageSpeed.units'].astype('string').fillna('', inplace=False)
df_resultados_corrida['vr_tempo'] = df_resultados_gerais['FastestLap.Time.time'].astype('string').fillna('', inplace=False)
df_resultados_corrida['vr_volta'] = df_resultados_gerais['FastestLap.lap'].astype('string').fillna('', inplace=False)
df_resultados_corrida['vr_posicao'] = df_resultados_gerais['FastestLap.rank'].astype('string').fillna('', inplace=False)

df_resultados_corrida

,temporada,etapa,piloto,construtor,numero,largada,voltas,posicao,posicao_t,pontos,situacao,tempo_mili,tempo,vr_media_velocidade,vr_media_unidade,vr_tempo,vr_volta,vr_posicao
0,1950,1,farina,alfa,2,1,70,1,1,9.0,Finished,8003600,2:13:23.6,0.000,,,,
1,1950,1,fagioli,alfa,3,2,70,2,2,6.0,Finished,8006200,+2.6,0.000,,,,
2,1950,1,reg_parnell,alfa,4,4,70,3,3,4.0,Finished,8055600,+52.0,0.000,,,,
3,1950,1,cabantous,lago,14,6,68,4,4,3.0,+2 Laps,,,0.000,,,,
4,1950,1,rosier,lago,15,9,68,5,5,2.0,+2 Laps,,,0.000,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,2022,20,mick_schumacher,haas,47,15,70,16,16,0.0,+1 Lap,,,185.248,kph,1:23.641,30,19
25796,2022,20,kevin_magnussen,haas,20,19,70,17,17,0.0,+1 Lap,,,186.007,kph,1:23.300,49,15
25797,2022,20,latifi,williams,6,18,69,18,18,0.0,+2 Laps,,,185.098,kph,1:23.709,56,20
25798,2022,20,alonso,alpine,14,9,63,19,19,0.0,Engine,,,186.981,kph,1:22.866,48,11


In [ ]:
df_resultados_corrida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25800 entries, 0 to 25799
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   temporada            25800 non-null  int64  
 1   etapa                25800 non-null  int64  
 2   piloto               25800 non-null  string 
 3   construtor           25800 non-null  string 
 4   numero               25800 non-null  string 
 5   largada              25800 non-null  int64  
 6   voltas               25800 non-null  int64  
 7   posicao              25800 non-null  int64  
 8   posicao_t            25800 non-null  string 
 9   pontos               25800 non-null  float64
 10  situacao             25800 non-null  string 
 11  tempo_mili           25800 non-null  string 
 12  tempo                25800 non-null  string 
 13  vr_media_velocidade  25800 non-null  float64
 14  vr_media_unidade     25800 non-null  string 
 15  vr_tempo             25800 non-null 

##### Tratamento dos campos

In [ ]:
# Tratando informações NULL
df_resultados_corrida = df_resultados_corrida.fillna('')

In [ ]:
# Todas informações em maisculo
df_resultados_corrida = df_resultados_corrida.apply(lambda x: x.astype(str).str.upper())
df_resultados_corrida['piloto'] = df_resultados_corrida['piloto'].astype(str).str.lower()
df_resultados_corrida['construtor'] = df_resultados_corrida['construtor'].astype(str).str.lower()

In [ ]:
# Tipando os campos
df_resultados_corrida['temporada'] = df_resultados_corrida['temporada'].astype(int)
df_resultados_corrida['etapa'] = df_resultados_corrida['etapa'].astype(int)

df_resultados_corrida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25800 entries, 0 to 25799
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   temporada            25800 non-null  int64 
 1   etapa                25800 non-null  int64 
 2   piloto               25800 non-null  object
 3   construtor           25800 non-null  object
 4   numero               25800 non-null  object
 5   largada              25800 non-null  object
 6   voltas               25800 non-null  object
 7   posicao              25800 non-null  object
 8   posicao_t            25800 non-null  object
 9   pontos               25800 non-null  object
 10  situacao             25800 non-null  object
 11  tempo_mili           25800 non-null  object
 12  tempo                25800 non-null  object
 13  vr_media_velocidade  25800 non-null  object
 14  vr_media_unidade     25800 non-null  object
 15  vr_tempo             25800 non-null  object
 16  vr_v

##### Obtendo chaves estrangeiras

In [ ]:
# Obtendo ID da Temporada
df_resultados_corrida = df_resultados_corrida.merge(df_temporadas[['temporada','temporada_id']],how='inner', on='temporada')
df_resultados_corrida


,temporada,etapa,piloto,construtor,numero,largada,voltas,posicao,posicao_t,pontos,situacao,tempo_mili,tempo,vr_media_velocidade,vr_media_unidade,vr_tempo,vr_volta,vr_posicao,temporada_id
0,1950,1,farina,alfa,2,1,70,1,1,9.0,FINISHED,8003600,2:13:23.6,0.0,,,,,1
1,1950,1,fagioli,alfa,3,2,70,2,2,6.0,FINISHED,8006200,+2.6,0.0,,,,,1
2,1950,1,reg_parnell,alfa,4,4,70,3,3,4.0,FINISHED,8055600,+52.0,0.0,,,,,1
3,1950,1,cabantous,lago,14,6,68,4,4,3.0,+2 LAPS,,,0.0,,,,,1
4,1950,1,rosier,lago,15,9,68,5,5,2.0,+2 LAPS,,,0.0,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,2022,20,mick_schumacher,haas,47,15,70,16,16,0.0,+1 LAP,,,185.248,KPH,1:23.641,30,19,73
25796,2022,20,kevin_magnussen,haas,20,19,70,17,17,0.0,+1 LAP,,,186.007,KPH,1:23.300,49,15,73
25797,2022,20,latifi,williams,6,18,69,18,18,0.0,+2 LAPS,,,185.098,KPH,1:23.709,56,20,73
25798,2022,20,alonso,alpine,14,9,63,19,19,0.0,ENGINE,,,186.981,KPH,1:22.866,48,11,73


In [ ]:
# Obtendo ID da etapa
df_resultados_corrida = df_resultados_corrida.merge(df_etapas[['etapa','temporada_id','etapa_id']],how='inner', on=['etapa','temporada_id'])
df_resultados_corrida

,temporada,etapa,piloto,construtor,numero,largada,voltas,posicao,posicao_t,pontos,situacao,tempo_mili,tempo,vr_media_velocidade,vr_media_unidade,vr_tempo,vr_volta,vr_posicao,temporada_id,etapa_id
0,1950,1,farina,alfa,2,1,70,1,1,9.0,FINISHED,8003600,2:13:23.6,0.0,,,,,1,1
1,1950,1,fagioli,alfa,3,2,70,2,2,6.0,FINISHED,8006200,+2.6,0.0,,,,,1,1
2,1950,1,reg_parnell,alfa,4,4,70,3,3,4.0,FINISHED,8055600,+52.0,0.0,,,,,1,1
3,1950,1,cabantous,lago,14,6,68,4,4,3.0,+2 LAPS,,,0.0,,,,,1,1
4,1950,1,rosier,lago,15,9,68,5,5,2.0,+2 LAPS,,,0.0,,,,,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,2022,20,mick_schumacher,haas,47,15,70,16,16,0.0,+1 LAP,,,185.248,KPH,1:23.641,30,19,73,1077
25796,2022,20,kevin_magnussen,haas,20,19,70,17,17,0.0,+1 LAP,,,186.007,KPH,1:23.300,49,15,73,1077
25797,2022,20,latifi,williams,6,18,69,18,18,0.0,+2 LAPS,,,185.098,KPH,1:23.709,56,20,73,1077
25798,2022,20,alonso,alpine,14,9,63,19,19,0.0,ENGINE,,,186.981,KPH,1:22.866,48,11,73,1077


In [ ]:
# Obtendo ID da Situacao
df_resultados_corrida = df_resultados_corrida.merge(df_situacao[['situacao','situacao_id']],how='left', on='situacao')
df_resultados_corrida

,temporada,etapa,piloto,construtor,numero,largada,voltas,posicao,posicao_t,pontos,situacao,tempo_mili,tempo,vr_media_velocidade,vr_media_unidade,vr_tempo,vr_volta,vr_posicao,temporada_id,etapa_id,situacao_id
0,1950,1,farina,alfa,2,1,70,1,1,9.0,FINISHED,8003600,2:13:23.6,0.0,,,,,1,1,1
1,1950,1,fagioli,alfa,3,2,70,2,2,6.0,FINISHED,8006200,+2.6,0.0,,,,,1,1,1
2,1950,1,reg_parnell,alfa,4,4,70,3,3,4.0,FINISHED,8055600,+52.0,0.0,,,,,1,1,1
3,1950,1,cabantous,lago,14,6,68,4,4,3.0,+2 LAPS,,,0.0,,,,,1,1,2
4,1950,1,rosier,lago,15,9,68,5,5,2.0,+2 LAPS,,,0.0,,,,,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,2022,20,mick_schumacher,haas,47,15,70,16,16,0.0,+1 LAP,,,185.248,KPH,1:23.641,30,19,73,1077,15
25796,2022,20,kevin_magnussen,haas,20,19,70,17,17,0.0,+1 LAP,,,186.007,KPH,1:23.300,49,15,73,1077,15
25797,2022,20,latifi,williams,6,18,69,18,18,0.0,+2 LAPS,,,185.098,KPH,1:23.709,56,20,73,1077,2
25798,2022,20,alonso,alpine,14,9,63,19,19,0.0,ENGINE,,,186.981,KPH,1:22.866,48,11,73,1077,9


In [ ]:
# Obtendo ID da piloto
df_resultados_corrida = df_resultados_corrida.merge(df_pilotos[['piloto','piloto_id']],how='left', on='piloto')
df_resultados_corrida

,temporada,etapa,piloto,construtor,numero,largada,voltas,posicao,posicao_t,pontos,situacao,tempo_mili,tempo,vr_media_velocidade,vr_media_unidade,vr_tempo,vr_volta,vr_posicao,temporada_id,etapa_id,situacao_id,piloto_id
0,1950,1,farina,alfa,2,1,70,1,1,9.0,FINISHED,8003600,2:13:23.6,0.0,,,,,1,1,1,1
1,1950,1,fagioli,alfa,3,2,70,2,2,6.0,FINISHED,8006200,+2.6,0.0,,,,,1,1,1,2
2,1950,1,reg_parnell,alfa,4,4,70,3,3,4.0,FINISHED,8055600,+52.0,0.0,,,,,1,1,1,3
3,1950,1,cabantous,lago,14,6,68,4,4,3.0,+2 LAPS,,,0.0,,,,,1,1,2,4
4,1950,1,rosier,lago,15,9,68,5,5,2.0,+2 LAPS,,,0.0,,,,,1,1,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,2022,20,mick_schumacher,haas,47,15,70,16,16,0.0,+1 LAP,,,185.248,KPH,1:23.641,30,19,73,1077,15,852
25796,2022,20,kevin_magnussen,haas,20,19,70,17,17,0.0,+1 LAP,,,186.007,KPH,1:23.300,49,15,73,1077,15,824
25797,2022,20,latifi,williams,6,18,69,18,18,0.0,+2 LAPS,,,185.098,KPH,1:23.709,56,20,73,1077,2,848
25798,2022,20,alonso,alpine,14,9,63,19,19,0.0,ENGINE,,,186.981,KPH,1:22.866,48,11,73,1077,9,763


In [ ]:
# Obtendo ID da construtor
df_resultados_corrida = df_resultados_corrida.merge(df_construtores[['construtor','construtor_id']],how='left', on='construtor')
df_resultados_corrida

,temporada,etapa,piloto,construtor,numero,largada,voltas,posicao,posicao_t,pontos,situacao,tempo_mili,tempo,vr_media_velocidade,vr_media_unidade,vr_tempo,vr_volta,vr_posicao,temporada_id,etapa_id,situacao_id,piloto_id,construtor_id
0,1950,1,farina,alfa,2,1,70,1,1,9.0,FINISHED,8003600,2:13:23.6,0.0,,,,,1,1,1,1,1
1,1950,1,fagioli,alfa,3,2,70,2,2,6.0,FINISHED,8006200,+2.6,0.0,,,,,1,1,1,2,1
2,1950,1,reg_parnell,alfa,4,4,70,3,3,4.0,FINISHED,8055600,+52.0,0.0,,,,,1,1,1,3,1
3,1950,1,cabantous,lago,14,6,68,4,4,3.0,+2 LAPS,,,0.0,,,,,1,1,2,4,2
4,1950,1,rosier,lago,15,9,68,5,5,2.0,+2 LAPS,,,0.0,,,,,1,1,2,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,2022,20,mick_schumacher,haas,47,15,70,16,16,0.0,+1 LAP,,,185.248,KPH,1:23.641,30,19,73,1077,15,852,207
25796,2022,20,kevin_magnussen,haas,20,19,70,17,17,0.0,+1 LAP,,,186.007,KPH,1:23.300,49,15,73,1077,15,824,207
25797,2022,20,latifi,williams,6,18,69,18,18,0.0,+2 LAPS,,,185.098,KPH,1:23.709,56,20,73,1077,2,848,142
25798,2022,20,alonso,alpine,14,9,63,19,19,0.0,ENGINE,,,186.981,KPH,1:22.866,48,11,73,1077,9,763,210


In [ ]:

df_resultados_corrida

,temporada,etapa,piloto,construtor,numero,largada,voltas,posicao,posicao_t,pontos,situacao,tempo_mili,tempo,vr_media_velocidade,vr_media_unidade,vr_tempo,vr_volta,vr_posicao,temporada_id,etapa_id,situacao_id,piloto_id,construtor_id
0,1950,1,farina,alfa,2,1,70,1,1,9.0,FINISHED,8003600,2:13:23.6,0.0,,,,,1,1,1,1,1
1,1950,1,fagioli,alfa,3,2,70,2,2,6.0,FINISHED,8006200,+2.6,0.0,,,,,1,1,1,2,1
2,1950,1,reg_parnell,alfa,4,4,70,3,3,4.0,FINISHED,8055600,+52.0,0.0,,,,,1,1,1,3,1
3,1950,1,cabantous,lago,14,6,68,4,4,3.0,+2 LAPS,,,0.0,,,,,1,1,2,4,2
4,1950,1,rosier,lago,15,9,68,5,5,2.0,+2 LAPS,,,0.0,,,,,1,1,2,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,2022,20,mick_schumacher,haas,47,15,70,16,16,0.0,+1 LAP,,,185.248,KPH,1:23.641,30,19,73,1077,15,852,207
25796,2022,20,kevin_magnussen,haas,20,19,70,17,17,0.0,+1 LAP,,,186.007,KPH,1:23.300,49,15,73,1077,15,824,207
25797,2022,20,latifi,williams,6,18,69,18,18,0.0,+2 LAPS,,,185.098,KPH,1:23.709,56,20,73,1077,2,848,142
25798,2022,20,alonso,alpine,14,9,63,19,19,0.0,ENGINE,,,186.981,KPH,1:22.866,48,11,73,1077,9,763,210


##### Criando campo chave

In [ ]:
df_resultados_corrida.reset_index(drop=True, inplace=True)
df_resultados_corrida['resultado_id'] = df_resultados_corrida.index + 1

##### Organizando colunas

In [ ]:
# Organizando Colunas
df_resultados_corrida = df_resultados_corrida[['resultado_id','temporada_id','etapa_id','piloto_id','construtor_id','situacao_id','numero','voltas','largada','posicao','posicao_t','pontos','tempo_mili','vr_tempo','vr_posicao','vr_volta','vr_media_velocidade']]
df_resultados_corrida

,resultado_id,temporada_id,etapa_id,piloto_id,construtor_id,situacao_id,numero,voltas,largada,posicao,posicao_t,pontos,tempo_mili,vr_tempo,vr_posicao,vr_volta,vr_media_velocidade
0,1,1,1,1,1,1,2,70,1,1,1,9.0,8003600,,,,0.0
1,2,1,1,2,1,1,3,70,2,2,2,6.0,8006200,,,,0.0
2,3,1,1,3,1,1,4,70,4,3,3,4.0,8055600,,,,0.0
3,4,1,1,4,2,2,14,68,6,4,4,3.0,,,,,0.0
4,5,1,1,5,2,2,15,68,9,5,5,2.0,,,,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,25796,73,1077,852,207,15,47,70,15,16,16,0.0,,1:23.641,19,30,185.248
25796,25797,73,1077,824,207,15,20,70,19,17,17,0.0,,1:23.300,15,49,186.007
25797,25798,73,1077,848,142,2,6,69,18,18,18,0.0,,1:23.709,20,56,185.098
25798,25799,73,1077,763,210,9,14,63,9,19,19,0.0,,1:22.866,11,48,186.981


In [ ]:
# Verificando resultado
df_resultados_corrida.info()
df_resultados_corrida

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25800 entries, 0 to 25799
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   resultado_id         25800 non-null  int64 
 1   temporada_id         25800 non-null  int64 
 2   etapa_id             25800 non-null  int64 
 3   piloto_id            25800 non-null  int64 
 4   construtor_id        25800 non-null  int64 
 5   situacao_id          25800 non-null  int64 
 6   numero               25800 non-null  object
 7   voltas               25800 non-null  object
 8   largada              25800 non-null  object
 9   posicao              25800 non-null  object
 10  posicao_t            25800 non-null  object
 11  pontos               25800 non-null  object
 12  tempo_mili           25800 non-null  object
 13  vr_tempo             25800 non-null  object
 14  vr_posicao           25800 non-null  object
 15  vr_volta             25800 non-null  object
 16  vr_m

,resultado_id,temporada_id,etapa_id,piloto_id,construtor_id,situacao_id,numero,voltas,largada,posicao,posicao_t,pontos,tempo_mili,vr_tempo,vr_posicao,vr_volta,vr_media_velocidade
0,1,1,1,1,1,1,2,70,1,1,1,9.0,8003600,,,,0.0
1,2,1,1,2,1,1,3,70,2,2,2,6.0,8006200,,,,0.0
2,3,1,1,3,1,1,4,70,4,3,3,4.0,8055600,,,,0.0
3,4,1,1,4,2,2,14,68,6,4,4,3.0,,,,,0.0
4,5,1,1,5,2,2,15,68,9,5,5,2.0,,,,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,25796,73,1077,852,207,15,47,70,15,16,16,0.0,,1:23.641,19,30,185.248
25796,25797,73,1077,824,207,15,20,70,19,17,17,0.0,,1:23.300,15,49,186.007
25797,25798,73,1077,848,142,2,6,69,18,18,18,0.0,,1:23.709,20,56,185.098
25798,25799,73,1077,763,210,9,14,63,9,19,19,0.0,,1:22.866,11,48,186.981


## Transformação - Fatos

### [Spark] Resultados Corrida

#### Criação dos DataSets

In [ ]:
# Criando Dataset Spark a partir do Dataset Pandas
spk_resultados = spark.createDataFrame(df_resultados_corrida)
spk_etapas = spark.createDataFrame(df_etapas)
spk_pilotos = spark.createDataFrame(df_pilotos)
spk_situacao = spark.createDataFrame(df_situacao)

In [ ]:
# View para consultas SQL
spk_resultados.createOrReplaceTempView("resultadosView")
spk_etapas.createOrReplaceTempView("etapasView")
spk_pilotos.createOrReplaceTempView("pilotosView")
spk_situacao.createOrReplaceTempView("situacaoView")

In [ ]:
# Query de companheiros Agrupados
spk_resultados = spark\
                    .sql("""
                        SELECT res.*,
                          etp.data etapa_data, 
                          plt.nascimento piloto_nascimento, 
                          collect_list(ceq.piloto_id) companheiro_id,  
                          collect_list(ceq.posicao) companheiro_posicao,      
                          collect_list(ceq.largada) companheiro_largada
                        FROM resultadosView res
                        INNER JOIN etapasView etp
                          on etp.etapa_id = res.etapa_id
                          and etp.temporada_id = res.temporada_id
                        LEFT JOIN pilotosView plt
                          on plt.piloto_id = res.piloto_id                              
                        LEFT JOIN resultadosView ceq 
                          on res.temporada_id = ceq.temporada_id
                          and res.etapa_id = ceq.etapa_id
                          and res.construtor_id = ceq.construtor_id
                          and res.piloto_id <> ceq.piloto_id
                        LEFT JOIN situacaoView sit
                          on res.situacao_id = sit.situacao_id
                        GROUP BY 
                          etp.data,
                          plt.nascimento,
                          res.resultado_id,
                          res.temporada_id,
                          res.etapa_id,
                          res.piloto_id,
                          res.construtor_id,
                          res.situacao_id,
                          res.numero,
                          res.voltas,
                          res.largada,
                          res.posicao,
                          res.posicao_t,
                          res.pontos,
                          res.tempo_mili,
                          res.vr_tempo,
                          res.vr_posicao,
                          res.vr_volta,
                          res.vr_media_velocidade
                        ORDER BY 
                          res.temporada_id,
                          res.etapa_id,
                          res.posicao
                    """)
spk_resultados.show()

+------------+------------+--------+---------+-------------+-----------+------+------+-------+-------+---------+------+----------+--------+----------+--------+-------------------+----------+-----------------+--------------------+--------------------+--------------------+
|resultado_id|temporada_id|etapa_id|piloto_id|construtor_id|situacao_id|numero|voltas|largada|posicao|posicao_t|pontos|tempo_mili|vr_tempo|vr_posicao|vr_volta|vr_media_velocidade|etapa_data|piloto_nascimento|      companheiro_id| companheiro_posicao| companheiro_largada|
+------------+------------+--------+---------+-------------+-----------+------+------+-------+-------+---------+------+----------+--------+----------+--------+-------------------+----------+-----------------+--------------------+--------------------+--------------------+
|           1|           1|       1|        1|            1|          1|     2|    70|      1|      1|        1|   9.0|   8003600|        |          |        |                0.0|1950-

In [ ]:
# Query para comparar resultados com companheiros de Equipe
spk_resultados_comp = spark\
                    .sql("""
                        SELECT
                          a.etapa_id,
                          a.piloto_id,
                          a.construtor_id,
                          a.posicao,
                          a.largada,
                          a.numero, 
                          b.piloto_id companheiro_id,  
                          b.posicao companheiro_posicao, 
                          b.largada companheiro_largada
                        FROM resultadosView a
                        LEFT JOIN resultadosView b 
                          on a.etapa_id = b.etapa_id
                          and a.construtor_id = b.construtor_id
                          and a.piloto_id <> b.piloto_id                          
                        ORDER BY a.temporada_id,a.etapa_id, a.posicao
                    """)

spk_resultados_comp

DataFrame[etapa_id: bigint, piloto_id: bigint, construtor_id: bigint, posicao: string, largada: string, numero: string, companheiro_id: bigint, companheiro_posicao: string, companheiro_largada: string]

#### Tratamentos

In [ ]:
spk_resultados.printSchema()
spk_resultados.count()

root
 |-- resultado_id: long (nullable = true)
 |-- temporada_id: long (nullable = true)
 |-- etapa_id: long (nullable = true)
 |-- piloto_id: long (nullable = true)
 |-- construtor_id: long (nullable = true)
 |-- situacao_id: long (nullable = true)
 |-- numero: string (nullable = true)
 |-- voltas: string (nullable = true)
 |-- largada: string (nullable = true)
 |-- posicao: string (nullable = true)
 |-- posicao_t: string (nullable = true)
 |-- pontos: string (nullable = true)
 |-- tempo_mili: string (nullable = true)
 |-- vr_tempo: string (nullable = true)
 |-- vr_posicao: string (nullable = true)
 |-- vr_volta: string (nullable = true)
 |-- vr_media_velocidade: string (nullable = true)
 |-- etapa_data: string (nullable = true)
 |-- piloto_nascimento: string (nullable = true)
 |-- companheiro_id: array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- companheiro_posicao: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- companheiro_l

25800

In [ ]:
# Tratamento dos Campos
spk_resultados = spk_resultados\
                    .withColumn("companheiro_id", concat_ws(",",col("companheiro_id")))\
                    .withColumn("companheiro_posicao", concat_ws(",",col("companheiro_posicao")))\
                    .withColumn("companheiro_largada", concat_ws(",",col("companheiro_largada")))\
                    .withColumn('etapa_data', f.to_date(spk_resultados.etapa_data.cast(StringType()),'yyyy-MM-dd'))\
                    .withColumn('piloto_nascimento', f.to_date(spk_resultados.piloto_nascimento.cast(StringType()),'yyyy-MM-dd'))\
                    .withColumn('idade_dias', datediff(col('etapa_data'),col('piloto_nascimento')).cast('int'))

                   
                    # .withColumn("datesDiff", datediff(col('corrida_data'),col('piloto_nascimento'))) \
                    # .withColumn("montsDiff", months_between(col('corrida_data'),col('piloto_nascimento'))) \
                    # .withColumn("montsDiff_round",round(months_between(col('corrida_data'),col('piloto_nascimento')),2)) \
                    # .withColumn("yearsDiff",months_between(col('corrida_data'),col('piloto_nascimento'))/lit(12)) \
                    # .withColumn("yearsDiff_round",round(months_between(col('corrida_data'),col('piloto_nascimento'))/lit(12),2)) 
                                                
spk_resultados.toPandas()

,resultado_id,temporada_id,etapa_id,piloto_id,construtor_id,situacao_id,numero,voltas,largada,posicao,posicao_t,pontos,tempo_mili,vr_tempo,vr_posicao,vr_volta,vr_media_velocidade,etapa_data,piloto_nascimento,companheiro_id,companheiro_posicao,companheiro_largada,idade_dias
0,1,1,1,1,1,1,2,70,1,1,1,9.0,8003600,,,,0.0,1950-05-13,1906-10-30,"2,3,13","2,3,12","2,4,3",15901
1,10,1,1,10,4,5,10,64,20,10,10,0.0,,,,,0.0,1950-05-13,1915-01-28,"9,11,15,16,18,19","9,10,14,15,17,18","16,20,5,18,8,11",12889
2,11,1,1,11,4,5,10,64,20,10,10,0.0,,,,,0.0,1950-05-13,1915-10-26,"9,10,15,16,18,19","9,10,14,15,17,18","16,20,5,18,8,11",12618
3,12,1,1,12,2,5,18,64,21,11,11,0.0,,,,,0.0,1950-05-13,1916-08-11,"4,5,8,20","4,5,8,19","6,9,14,7",12328
4,13,1,1,13,1,6,1,62,3,12,R,0.0,,,,,0.0,1950-05-13,1911-06-24,"1,2,3","1,2,3","1,2,4",14203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,25785,73,1077,830,6,1,55,71,5,5,5,10.0,5974852,1:22.199,6,32,188.498,2022-10-30,1994-09-01,843,6,7,10286
25796,25786,73,1077,843,6,1,16,71,7,6,6,8.0,5985503,1:22.603,10,48,187.576,2022-10-30,1997-10-16,830,5,5,9145
25797,25787,73,1077,816,117,15,3,70,11,7,7,6.0,,1:22.022,3,48,188.905,2022-10-30,1989-07-01,845,9,8,12174
25798,25788,73,1077,838,210,15,31,70,10,8,8,4.0,,1:23.279,14,48,186.054,2022-10-30,1996-09-17,763,19,9,9539


In [ ]:
spk_resultados.printSchema()
spk_resultados.count()

root
 |-- resultado_id: long (nullable = true)
 |-- temporada_id: long (nullable = true)
 |-- etapa_id: long (nullable = true)
 |-- piloto_id: long (nullable = true)
 |-- construtor_id: long (nullable = true)
 |-- situacao_id: long (nullable = true)
 |-- numero: string (nullable = true)
 |-- voltas: string (nullable = true)
 |-- largada: string (nullable = true)
 |-- posicao: string (nullable = true)
 |-- posicao_t: string (nullable = true)
 |-- pontos: string (nullable = true)
 |-- tempo_mili: string (nullable = true)
 |-- vr_tempo: string (nullable = true)
 |-- vr_posicao: string (nullable = true)
 |-- vr_volta: string (nullable = true)
 |-- vr_media_velocidade: string (nullable = true)
 |-- etapa_data: date (nullable = true)
 |-- piloto_nascimento: date (nullable = true)
 |-- companheiro_id: string (nullable = false)
 |-- companheiro_posicao: string (nullable = false)
 |-- companheiro_largada: string (nullable = false)
 |-- idade_dias: integer (nullable = true)



25800

#### Fatos

##### Metricas do Piloto x Companheiro de Equipe

In [ ]:
# Preparação dos Novos Campos Calculados
posicao_comp = when(col("posicao")<col('companheiro_posicao'), 1).otherwise(0)
largada_comp = when(col("largada")<col('companheiro_largada'), 1).otherwise(0)

# Aplicação dos Novos Campos Calculados
spk_resultados_comp = spk_resultados_comp\
                        .withColumn('posicao_comp', posicao_comp)\
                        .withColumn('largada_comp', largada_comp)

In [ ]:
# Agregação do Fatos
spk_resultados_comp = spk_resultados_comp\
                        .groupBy( 'etapa_id', 'numero', 'piloto_id', 'construtor_id')\
                        .agg(
                            f.sum('posicao_comp').alias('posicao_comp'),
                            f.sum('largada_comp').alias('largada_comp')
                        )\
                        .orderBy( 'etapa_id', 'piloto_id', 'construtor_id', ascending=True)

spk_resultados_comp.count()


25800

##### Metricas do Piloto

In [ ]:
vitoria = when(col("posicao")==1, 1).otherwise(0)
podio = when(col("posicao")<=3, 1).otherwise(0)
pole = when(col("largada")==1, 1).otherwise(0)
abandono = when(col("posicao_t").cast("int").isNull().alias("Value "),1).otherwise(0)
volta_rapida = when(col("vr_posicao")==1, 1).otherwise(0)

spk_resultados = spk_resultados\
                    .withColumn('vitoria', vitoria)\
                    .withColumn('podio', podio)\
                    .withColumn('pole', pole)\
                    .withColumn('abandono', abandono)\
                    .withColumn('volta_rapida', volta_rapida)

In [ ]:
print(spk_resultados.count())
print(spk_resultados_comp.count())

25800
25800


#### Join dos DataSets

In [ ]:
spk_resultados = spk_resultados.join(spk_resultados_comp,['etapa_id','numero','piloto_id','construtor_id'], how='inner').orderBy('etapa_id', 'piloto_id', 'construtor_id', ascending=True)
spk_resultados.toPandas()

,etapa_id,numero,piloto_id,construtor_id,resultado_id,temporada_id,situacao_id,voltas,largada,posicao,posicao_t,pontos,tempo_mili,vr_tempo,vr_posicao,vr_volta,vr_media_velocidade,etapa_data,piloto_nascimento,companheiro_id,companheiro_posicao,companheiro_largada,idade_dias,vitoria,podio,pole,abandono,volta_rapida,posicao_comp,largada_comp
0,1,2,1,1,1,1,1,70,1,1,1,9.0,8003600,,,,0.0,1950-05-13,1906-10-30,"2,3,13","2,3,12","2,4,3",15901,1,1,1,0,0,3,3
1,1,3,2,1,2,1,1,70,2,2,2,6.0,8006200,,,,0.0,1950-05-13,1898-06-09,"1,3,13","1,3,12","1,4,3",18965,0,1,0,0,0,1,2
2,1,4,3,1,3,1,1,70,4,3,3,4.0,8055600,,,,0.0,1950-05-13,1911-07-02,"1,2,13","1,2,12","1,2,3",14195,0,1,0,0,0,0,0
3,1,14,4,2,4,1,2,68,6,4,4,3.0,,,,,0.0,1950-05-13,1904-10-08,"5,8,12,20","5,8,11,19","9,14,21,7",16653,0,0,0,0,0,2,2
4,1,15,5,2,5,1,2,68,9,5,5,2.0,,,,,0.0,1950-05-13,1905-11-05,"4,8,12,20","4,8,11,19","6,14,21,7",16260,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25795,1077,63,847,48,25784,73,1,71,2,4,4,13.0,5966160,1:20.153,1,71,193.31,2022-10-30,1998-02-15,795,2,3,9023,0,0,0,0,1,0,1
25796,1077,6,848,142,25798,73,2,69,18,18,18,0.0,,1:23.709,20,56,185.098,2022-10-30,1995-06-29,846,12,17,9985,0,0,0,0,0,0,0
25797,1077,22,851,209,25800,73,128,50,13,20,R,0.0,,1:23.403,18,48,185.777,2022-10-30,2000-05-11,841,11,14,8207,0,0,0,1,0,0,1
25798,1077,47,852,207,25796,73,15,70,15,16,16,0.0,,1:23.641,19,30,185.248,2022-10-30,1999-03-22,824,17,19,8623,0,0,0,0,0,1,1


In [ ]:
spk_resultados.coalesce(1).write.csv('/content/drive/MyDrive/Projetos/BI/F1/resultados.csv',sep=',',mode='overwrite',header=True)


KeyboardInterrupt: ignored

In [ ]:
spk_resultados.toPandas().to_csv('/content/drive/MyDrive/Projetos/BI/F1/dw/resultados.csv',sep=',')

## Finalizando Spark

In [ ]:
#spark.stop()